# Libraries

In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Settings

In [2]:
# Path
image_path = '../02_Data/face_age' # Added path to gitingnore, you will have to add data to this path

# Image Manipulation

In [3]:
def load_file_paths(folder_path):
    file_paths = []
    ages = []
    for folder in os.listdir(folder_path):
        if os.path.isdir(os.path.join(folder_path, folder)):
            age = int(folder)
            for file in os.listdir(os.path.join(folder_path, folder)):
                file_path = os.path.join(folder_path, folder, file)
                file_paths.append(file_path)
                ages.append(age)
    return file_paths, ages

folder_path = image_path
file_paths, ages = load_file_paths(folder_path)

In [4]:
file_paths_train, file_paths_test, ages_train, ages_test = train_test_split(file_paths, ages, test_size=0.2, random_state=42)

In [5]:
from keras.preprocessing.image import ImageDataGenerator

def custom_data_generator(file_paths, ages, batch_size, img_size, augment=False):
    data_gen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20 if augment else 0,
        width_shift_range=0.2 if augment else 0,
        height_shift_range=0.2 if augment else 0,
        shear_range=0.2 if augment else 0,
        zoom_range=0.2 if augment else 0,
        horizontal_flip=True if augment else False,
        fill_mode='nearest'
    )
    
    while True:
        idx = np.random.permutation(len(file_paths))
        for batch_start in range(0, len(file_paths), batch_size):
            batch_idx = idx[batch_start:batch_start+batch_size]
            batch_file_paths = [file_paths[i] for i in batch_idx]
            batch_ages = [ages[i] for i in batch_idx]

            batch_images = []
            for file_path in batch_file_paths:
                img = Image.open(file_path)
                img = img.resize(img_size)
                img = np.array(img)
                batch_images.append(img)
                
            batch_images = np.array(batch_images)
            batch_ages = np.array(batch_ages)
            batch_images_augmented = data_gen.flow(batch_images, batch_ages, batch_size=batch_size, shuffle=False).next()
            yield batch_images_augmented

In [6]:
batch_size = 32
img_size = (200, 200)
train_generator = custom_data_generator(file_paths_train, ages_train, batch_size, img_size, augment=True)
test_generator = custom_data_generator(file_paths_test, ages_test, batch_size, img_size, augment=False)

In [7]:
def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

input_shape = (img_size[0], img_size[1], 3)
model = create_model(input_shape)

In [8]:
steps_per_epoch = len(file_paths_train) // batch_size
validation_steps = len(file_paths_test) // batch_size
epochs = 30

history = model.fit(train_generator, validation_data=test_generator, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, verbose=1)

Epoch 1/30
244/244 [==============================] - 74s 284ms/step - loss: 1233.7545 - mae: 27.8761 - val_loss: 1333.6470 - val_mae: 29.4389
Epoch 2/30
244/244 [==============================] - 58s 239ms/step - loss: 731.2318 - mae: 21.3295 - val_loss: 1485.6138 - val_mae: 33.8352
Epoch 3/30
244/244 [==============================] - 58s 240ms/step - loss: 398.3294 - mae: 15.0317 - val_loss: 199.4479 - val_mae: 10.5144
Epoch 4/30
244/244 [==============================] - 60s 246ms/step - loss: 314.3793 - mae: 13.3964 - val_loss: 782.4519 - val_mae: 23.8049
Epoch 5/30
244/244 [==============================] - 66s 270ms/step - loss: 291.5349 - mae: 12.9078 - val_loss: 225.0082 - val_mae: 11.3470
Epoch 6/30
244/244 [==============================] - 67s 276ms/step - loss: 277.7637 - mae: 12.6512 - val_loss: 213.8882 - val_mae: 10.9707
Epoch 7/30
244/244 [==============================] - 66s 272ms/step - loss: 263.3974 - mae: 12.2080 - val_loss: 167.8331 - val_mae: 9.6949
Epoch 8/30


ResourceExhaustedError: Graph execution error:

2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  MemoryError: Unable to allocate 14.6 MiB for an array with shape (32, 200, 200, 3) and data type float32
Traceback (most recent call last):

  File "C:\Users\nuke2\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "C:\Users\nuke2\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\nuke2\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "c:\Users\nuke2\miniconda3\lib\site-packages\keras\engine\data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\nuke2\AppData\Local\Temp\ipykernel_4884\2759120128.py", line 31, in c

# 